In [61]:
%matplotlib notebook
import matplotlib.lines as lines
import matplotlib.transforms as mtransforms
import matplotlib.text as mtext
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from math import pi

In [62]:
class MyLine(lines.Line2D):
    def __init__(self, *args, **kwargs):
        # we'll update the position when the line data is set
        self.text = mtext.Text(0, 0, '')
        lines.Line2D.__init__(self, *args, **kwargs)

        # we can't access the label attr until *after* the line is
        # initiated
        self.text.set_text(self.get_label())

    def set_figure(self, figure):
        self.text.set_figure(figure)
        lines.Line2D.set_figure(self, figure)

    def set_axes(self, axes):
        self.text.set_axes(axes)
        lines.Line2D.set_axes(self, axes)

    def set_transform(self, transform):
        # 2 pixel offset
        texttrans = transform + mtransforms.Affine2D().translate(2, 2)
        self.text.set_transform(texttrans)
        lines.Line2D.set_transform(self, transform)

    def set_data(self, x, y):
        if len(x):
            self.text.set_position((x[-1], y[-1]))

        lines.Line2D.set_data(self, x, y)

    def draw(self, renderer):
        # draw my label at the end of the line with 2 pixel offset
        lines.Line2D.draw(self, renderer)
        self.text.draw(renderer)

In [63]:
def ColClear(string):
    while len(string) and not (string[0] >= "0" and string[0] <= "9"):
        string = string[1:]
    return string

In [64]:
def transform_data(file_name):
    tmp_data = pd.read_csv(file_name)
    tmp_data.to_csv("TMP")

    data = pd.read_csv("TMP", header=None)

    i = 0
    while (data.iloc[i][1][:4] != " PAL"):
        i += 1
    a = []
    for j in range(i + 1):
        a.append(j)
    data.drop(a, axis=0, inplace=True)

    tmp_tb = []
    for i in data[1]:
        tmp_tb.append(i.split())

    data = pd.DataFrame(tmp_tb)

    test = data[0]

    Col1Measure = "T"

    if data[0][0][0] == "M":
        Col1Measure = "M"

    data[0] = list(map(ColClear, test))
    data.drop(len(data) - 1, axis=0, inplace=True)

    data.rename(
        columns={0: Col1Measure, 1: "X", 2: "Y", 3: "Z", 4: "MAG", 5: "D", 6: "I", 7: "D_old", 8: "I_old", 9: "a95"},
        inplace=True)

    for i in data.columns:
        data[i] = pd.to_numeric(data[i])

    return data, Col1Measure

In [65]:
def draw_polar(system, file_name):

    data, Col1Measure=transform_data(file_name)
    fig = plt.figure()
    ax = plt.subplot(polar=True)

    employee = ["N", "E", "S", "W"]

    ax.set_thetagrids(range(0, 360, int(360 / len(employee))), employee, family="Times New Roman", weight=0)

    ax.set_rgrids([10, 20, 30, 40, 50, 60, 70, 80, 90], fmt="")

    ax.tick_params(axis="both", labelsize=26)
    ax.grid(color='k', linestyle='', linewidth=1, zorder=0.1)

    # Costyl axes

    for i in [0, 90, 180, 270]:
        for j in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
            ax.scatter(i/180*pi, j, s=1, alpha=1, c=(0, 0, 0), zorder=0.1)
    ax.scatter(0, 0, s=20, alpha=1, c=(0,0,0), zorder=0.11, marker="+")

    # Define arrays

    col_D = "D"
    col_I = "I"

    if system == "old":
        col_D = "D_old"
        col_I = "I_old"

    angles_neg = list()
    angles = list()
    values_neg = list()
    values = list()
    values_abs = list(abs(data[col_I]))
    angles_abs = list(abs(data[col_D]) / 180 * pi)

    for i in range(0, len(data[col_I]), 1):
        if data["I"][i] < 0:
            values_neg.append(abs(data[col_I][i]))
            angles_neg.append(data[col_D][i] / 180 * pi)
        else:
            values.append(data[col_I][i])
            angles.append(data[col_D][i] / 180 * pi)

    # Plot dots
    
    marker_style = dict(color=(0,0,0), linestyle=':', marker='o',
                    markersize=15, markerfacecoloralt='tab:gray')

    ax.scatter(angles_neg, values_neg, s=25, alpha=1, c=(1,1,1), zorder=1, edgecolors=(0,0,0), linewidth=0.5)
    ax.scatter(angles_abs, values_abs, s=25, alpha=1, c=(0,0,0), zorder=0.5)

    # ax.scatter(90/180*pi,50,linewidths=2.5,alpha=1,c=(1,0.5,1))

    ax.set_rmax(90)
    ax.set_rmin(0)

    # Plot lines

    line = MyLine(angles_abs, values_abs, color=(0,0,0), linewidth=0.5, zorder = 0.4)
    ax.add_line(line)

    # Plot labels

    if Col1Measure == "T":

        ax.text(angles_abs[0], values_abs[0], s=str(data["T"][0]) + '\xb0', ha='left', va='bottom', size="xx-large",
                family="Times New Roman")
        ax.text(angles_abs[len(angles_abs) - 1], values_abs[len(angles_abs) - 1],
                s=str(data["T"][len(angles_abs) - 1]) + '\xb0',
                ha='left', va='bottom', size="xx-large", family="Times New Roman")
    else:
        ax.text(angles_abs[0], values_abs[0], s=str(data["M"][0]) + "mT", ha='left', va='bottom', size="xx-large",
                family="Times New Roman")
        ax.text(angles_abs[len(angles_abs) - 1], values_abs[len(angles_abs) - 1],
                s=str(data["M"][len(angles_abs) - 1]) + "mT",
                ha='left', va='bottom', size="xx-large", family="Times New Roman")

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    #plt.show()
    return fig

In [95]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select File"
        self.icon = "square-o"
        self.style.button_color = "yellow"
        # Set on click behavior.
        self.on_click(self.select_file)

    @staticmethod
    def select_file(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.file = filedialog.askopenfilename(multiple=False)

        b.description = "File Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [96]:
choose_file = SelectFilesButton()
choose_file

SelectFilesButton(description='Select File', icon='square-o', style=ButtonStyle(button_color='yellow'))

In [97]:
#df, colname = transform_data(name+".pmd")
@interact
def correlations(system=list(["old", "new"])):
    fig = draw_polar(system, choose_file.file)

interactive(children=(Dropdown(description='system', options=('old', 'new'), value='old'), Output()), _dom_cla…

In [55]:
tmp_df=pd.DataFrame(df[colname])
tmp_df

,T
0,20
1,100
2,180
3,240
4,300
5,340
6,380
7,420
8,460
9,500
